In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

c:\Users\prave\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
class KSOM(nn.Module):
    def __init__(self, input_dim, output_dim, learning_rate, weights):
        super(KSOM, self).__init__()

        self.input_dim = input_dim
        self.output_dim = output_dim

        self.learning_rate = learning_rate
        
        self.weights = nn.Parameter(weights)

    def forward(self, x):
    
        x = x.view(-1, self.input_dim)
        distances = torch.sum((self.weights - x.unsqueeze(1)) ** 2, dim=-1)
        winner = torch.argmin(distances, dim=1)
        
        delta = x.unsqueeze(1)+self.learning_rate * (x.unsqueeze(1) - self.weights)
        self.weights.data[winner] += torch.sum(delta, dim=0)[winner]

        return winner

In [4]:
x = torch.tensor([0.0, 0.2, 0.1, 0.2, 0.0])

weights = torch.tensor([
    [1.0, 0.9, 0.7, 0.3, 0.2],
    [0.6, 0.7, 0.5, 0.4, 1.0],
])

In [5]:
ksom = KSOM(input_dim=5, output_dim=2, learning_rate=0.2, weights=weights)


winner = ksom(x)
print("Winning cluster unit:", winner)


print("Updated weights:", ksom.weights[winner])



Winning cluster unit: tensor([1])
Updated weights: tensor([[0.4800, 0.8000, 0.5200, 0.5600, 0.8000]], grad_fn=<IndexBackward>)


In [6]:
print("Updated learning rate:",0.5*ksom.learning_rate)

Updated learning rate: 0.1
